# Indonesian News Generator

Trained with RNN LSTM on PyTorch.

To do :
- Add more data, for validation and test
- Add more measurement to evaluate trained model (e.g. perplexity)
- Explore more criterion Loss Function
- Bayesian optimization on hyperparameter using Gaussian Process

## Preparation

In [1]:
# import library needed
import unidecode
import string
import random
import re
import time, math

import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
# read file into one big string
tdata = open('news_indo_edit.txt').read()

In [3]:
#define parameter for running
chunk_len = 60
batch_size = 30
all_characters = string.printable
n_char = len(all_characters)

n_epochs = 20
hidden_size = 120
n_layers = 3
lr = 1e-3
clip = 0.25

print_every = 200

In [4]:
data = ""
for char in tdata:
    if char in all_characters:
        data += char

In [5]:
def get_chunk(start_index, chunk_len, data):
    end_index = start_index + chunk_len + 1
    return data[start_index:end_index]

In [6]:
# turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

def get_tensor_pair(start_index, chunk_len, data):    
    chunk = get_chunk(start_index, chunk_len, data)
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [7]:
def generate_batch(start_index, chunk_len, batch_size, data):
    inps = torch.zeros(chunk_len,batch_size).long()
    targets = torch.zeros(chunk_len,batch_size).long()
    for i in range(batch_size):
        inp, target = get_tensor_pair(start_index, chunk_len, data)
        inps[:,i] = inp
        targets[:,i] = target
        start_index += chunk_len
    # uncomment to do this with CPU
    #return Variable(inps), Variable(targets)
    return Variable(inps.cuda()), Variable(targets.cuda())

## Build the Model

In [8]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0.2):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=dropout)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden, cell, batch_size, chunk_len):
        input = self.encoder(input)
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        output = self.decoder(output.view(batch_size * chunk_len,-1))
        return output, hidden, cell

    def init_hidden(self, batch_size):
        h0 = Variable(torch.zeros(n_layers, batch_size, hidden_size).cuda())
        c0 = Variable(torch.zeros(n_layers, batch_size, hidden_size).cuda())
        return h0, c0

## Evaluation

In [9]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden, cell = decoder.init_hidden(1)
    prime_input = char_tensor(prime_str)
    prime_input = Variable(prime_input.cuda())
    predicted = prime_str
    
    if len(prime_input) != 1:
        _, hidden, cell = decoder(prime_input[:-1].unsqueeze(1), hidden, cell, 1, len(prime_str)-1)
    
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden, cell = decoder(inp.unsqueeze(1), hidden, cell, 1, 1)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1, n_char).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0][0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)
        inp = Variable(inp.cuda())

    return predicted

## Training

In [10]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [11]:
resume = True

if resume:
    decoder = torch.load('saved_model/checkpoint_100.tar')
else:
    decoder = RNN(n_char, hidden_size, n_char, n_layers)
    
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [12]:
decoder.cuda()

RNN (
  (encoder): Embedding(100, 120)
  (lstm): LSTM(120, 120, num_layers=3, dropout=0.2)
  (decoder): Linear (120 -> 100)
)

In [13]:
def train(inp, target, hidden, cell):
    h0, c0 = decoder.init_hidden(batch_size)
    decoder.zero_grad()
    loss = 0

    output, hidden, cell = decoder(inp, hidden, cell, batch_size, chunk_len)
    loss = criterion(output.view(-1, n_char), target.view(-1))

    loss.backward()
    torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)
    for p in decoder.parameters():
        p.data.add_(-lr, p.grad.data)

    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
    decoder_optimizer.step()

    return loss.data[0], hidden, cell

In [14]:
def repackage_hidden(h, c):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data), Variable(c.data)
    else:
        return tuple(repackage_hidden(v) for v in h), tuple(repackage_hidden(u) for u in c)

In [117]:
start = time.time()
iters = 0
decoder.train()
for epoch in range(1, n_epochs + 1):
    start_index = 0
    hidden, cell = decoder.init_hidden(batch_size)
    while start_index + chunk_len * batch_size < len(data):
        inp, target = generate_batch(start_index, chunk_len, batch_size, data)
        hidden, cell = repackage_hidden(hidden, cell)
        loss, hidden, cell = train(inp, target, hidden, cell)
        
        if iters % print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, ((start_index) * 1.0 / len(data)) * 100, loss))
            decoder.eval()
            print(evaluate('KOMPAS.com', 300), '\n')
            decoder.train()

        iters += 1
        start_index += chunk_len * batch_size
    if(lr > 1e-14):
        lr /= 2
with open('saved_model/checkpoint_100.tar','w') as ckpt:
    torch.save(decoder, ckpt)

 0s (1 0%) 1.5127]
('KOMPAS.com - Satu ini.\nDia mengontorini melakukan sekarang semis mungkin, bersalah penjadi tidak negara di pada jadi orang merurikan mengambah setak dan ini berkapan.\nSaya menggunakan menyebut sekitar bagi kemenamanan dipersistarangan pemerintah di Kaburat resmi membandung.\nMana terangkak tersebut.\nSemampu me', '\n')
 31s (1 19%) 1.7419]
('KOMPAS.com - AS juga mendapatkan terbeda darikan kejuaraan di Desember mengatakan Gan sebuah politik seperti namun oleh Penting, Bandar Ferten Madrid (ES).\nBersubut yang dipada gransdingan dan penumpang dihasil di untuk menerima.\nSementara tersebut.\n"Kami tersebut digunakan, "jamta Polkadd, cetangan tahun dan', '\n')
 1s (1 39%) 1.5320]
('KOMPAS.com - Kapanlagi.com - Mereka trial.\n\nMerdeka.com - Kapanlagi.com - Pertama Sekan membuat keras mengelalu bersama ada Brahaya Selasa di Pusobaerana.\n"Kapala porsi jelas di sekitar malam sisi dari pasaran yang pintang sepertinya kami," tatpang, penting, termodi berpernah seperti b

## Result

In [5]:
decoder = torch.load('saved_model/checkpoint_100.tar')

In [44]:
decoder.cuda()

RNN (
  (encoder): Embedding(100, 120)
  (lstm): LSTM(120, 120, num_layers=3, dropout=0.5)
  (decoder): Linear (120 -> 100)
)

In [118]:
decoder.eval()

RNN (
  (encoder): Embedding(100, 120)
  (lstm): LSTM(120, 120, num_layers=3, dropout=0.2)
  (decoder): Linear (120 -> 100)
)

In [24]:
inc = 0.05
result = ""
result2 = ""
for i in range(10):
    result += evaluate('Kompas.com', 500, temperature=0.4+inc) + "\n \n"
    result2 += evaluate('Merdeka.com', 500, temperature=0.4+inc) + "\n \n"
    inc += 0.05

In [25]:
print result

Kompas.com - Kapanlagi.com - Pemain itu terbaik dari seharus di keras dari pasing pertandingan dengan selama di saja terjadi tidak memberikan untuk membuat lain demikian.
Dia mengenakan ke Netaria berdasarkan setelah tersebut dia sendiri ke Balan pernang di Tone pengangkutan lalu.
Sebelumnya di Kota Kamis Partai Kamis (20/2/2012).
"Kita meminta kendaraan itu, Anda adalah pertanyaan pembuatan dan pemain terbaik di sendiri karena akan mengatakan sebelum semua penyidik kekuatan itu tersebut diterima kendati 
 
Kompas.com - Riyang dan melalui sebelum mengetahui penyelenggaraan serangan penjara pertama dan tersebut dia terakhir.
"Saya merikat percaya tertah menyatakan selama bagi bagian dan terus menjadi pertama selurungan dengan perlaya sangat dari tersebut sebagai pada percaya ketua final apa tanjang pertama setambahan sebagai pemerintah dengan kelembaikan mengatakan kemain berada sedikit sebagai di Selasa (HAS) menarah di Motel pembeli dan salah bersama setiap di Bandung (24/1/2012).
Men

In [26]:
print result2

Merdeka.com - Kapanlagi.com - Pengamas Hernah dan pertama di lalu, semua tersebut di selama ke Pinggaran, mengatakan mengatakan ke Rasa kepada Anda berikut pertandingan membuat kemarin di banyak mereka tidak terbaik dan menggangikan akan mendapatkan pemerintah penjara ini tidak bertahan pertandingan pada setelah pertama berbangunan di ke kami di sementara di depan, ketiga siap pertama ini berlangsung sebelum mengenai kepada tersebut mengatakan kepada kepelayanan dan dalam kepada sebagai pertandingan itu me
 
Merdeka.com - Kepana Angga dan aku pemerintah setelah penyidikan pertama.
Sebelumnya sebagian dan sebagai di satu perusahaan di Malang memiliki pernyataan ini mengenai banyak akan beridangan pertama di politik itu memberikan ketika kembali mengatakan ke dari tersebut di pertama menyangka kembali menyelesaikan tahun dan memang bersama dan sekitar Senin (1/3/2012).
Sadar ada itu di dan tersebut sekitar mengenai saat tersebut.
"Dalam sama seberang kepada Soolo tersebut mengatakan penj